I decided to treat this as a classification problem by creating a new binary variable affair (did the woman have at least one affair?) and trying to predict the classification for each woman.


Dataset

The dataset I chose is the affairs dataset that comes with Statsmodels. It was derived from a survey of women in 1974 by Redbook magazine, in which married women were asked about their participation in extramarital affairs. More information about the study is available in a 1978 paper from the Journal of Political Economy.


Description of Variables

The dataset contains 6366 observations of 9 variables: <br>
rate_marriage: woman's rating of her marriage (1 = very poor, 5 = very good) <br>
age: woman's age <br>
yrs_married: number of years married <br>
children: number of children <br>
religious: woman's rating of how religious she is (1 = not religious, 4 = strongly religious) <br>
educ: level of education (9 = grade school, 12 = high school, 14 = some college, 16 = college graduate, 17 = some graduate school, 20 = advanced degree) <br>
occupation: woman's occupation (1 = student, 2 = farming/semi-skilled/unskilled, 3 = "white collar", 4 = 
teacher/nurse/writer/technician/skilled, 5 = managerial/business, 6 = professional with advanced degree) <br>
occupation_husb: husband's occupation (same coding as above) <br>
affairs: time spent in extra-marital affairs <br>


Code to loading data and modules:

import numpy as np <br>
import pandas as pd <br>
import statsmodels.api as sm <br>
import matplotlib.pyplot as plt <br>
from patsy import dmatrices <br>
from sklearn.linear_model import LogisticRegression <br>
from sklearn.cross_validation import train_test_split <br>
from sklearn import metrics <br>
from sklearn.cross_validation import cross_val_score <br>

dta = sm.datasets.fair.load_pandas().data <br>

#add "affair" column: 1 represents having affairs, 0 represents not  <br>
dta['affair'] = (dta.affairs > 0).astype(int) <br>
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \ religious + educ + C(occupation) + C(occupation_husb)', dta, return_type="dataframe") <br>
X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2', <br> 
'C(occupation)[T.3.0]':'occ_3', <br>
'C(occupation)[T.4.0]':'occ_4', <br>
'C(occupation)[T.5.0]':'occ_5', <br>
'C(occupation)[T.6.0]':'occ_6', <br>
'C(occupation_husb)[T.2.0]':'occ_husb_2', <br>
'C(occupation_husb)[T.3.0]':'occ_husb_3', <br>
'C(occupation_husb)[T.4.0]':'occ_husb_4', <br>
'C(occupation_husb)[T.5.0]':'occ_husb_5', <br>
'C(occupation_husb)[T.6.0]':'occ_husb_6'}) <br>

y = np.ravel(y)

In [1]:
#loading libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import dtale
%matplotlib inline

In [2]:
#importing data

dta = sm.datasets.fair.load_pandas().data
dta['affair'] = (dta['affairs'] > 0).astype(int)
dta.drop(columns=['affairs'],inplace=True)
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,1


In [3]:
#quick analysis using dtale
dtale.show(dta)

#Observations
#1. No outliers found
#2. No. of children for 203 rows listed as 5.5; Continued as is, assuming it to be 5

In [4]:
#creating duplicate dataframe by splitting categorical columns

rate_m = pd.get_dummies(dta['rate_marriage'],prefix='RM')
children = pd.get_dummies(dta['children'],prefix='children')
religious = pd.get_dummies(dta['religious'],prefix='religious')
educ = pd.get_dummies(dta['educ'],prefix='educ')
occup = pd.get_dummies(dta['occupation'],prefix='occup')
occup_hus = pd.get_dummies(dta['occupation_husb'],prefix='occup_hus')
dta1 = pd.concat([dta,rate_m,children,religious,educ,occup,occup_hus],axis=1)
dta1.drop(columns=['rate_marriage','children','religious','educ','occupation','occupation_husb'],inplace=True)
dta1.head()

,age,yrs_married,affair,RM_1.0,RM_2.0,RM_3.0,RM_4.0,RM_5.0,children_0.0,children_1.0,...,occup_3.0,occup_4.0,occup_5.0,occup_6.0,occup_hus_1.0,occup_hus_2.0,occup_hus_3.0,occup_hus_4.0,occup_hus_5.0,occup_hus_6.0
0,32.0,9.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,27.0,13.0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,22.0,2.5,1,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,1,0
3,37.0,16.5,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,27.0,9.0,1,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0


In [5]:
#creating a model on original dataframe, validating its score and performing feature selection using RFE to see if the model 
#accuracy can be preserved with lesser features

X = dta.drop(columns=['affair'])
y = dta['affair']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
lr = LogisticRegression()
lr.fit(X_train,y_train)
kf = KFold(shuffle=True, n_splits=5)
cv_results = cross_val_score(lr, X, y, cv=kf, scoring='accuracy')
y_pred = lr.predict(X_test)

print(cv_results)
print(metrics.accuracy_score(y_test,y_pred))

rfe = RFE(estimator=lr,n_features_to_select=4)
rfe.fit(X,y)
X_new = X[X.columns[rfe.get_support(1)]]
X_train_new,X_test_new,y_train_new,y_test_new = train_test_split(X_new,y,test_size=0.25,random_state=42)
lr_new = LogisticRegression()
lr_new.fit(X_train_new,y_train_new)
kf_new = KFold(shuffle=True, n_splits=5)
cv_results_new = cross_val_score(lr_new, X_new, y, cv=kf_new, scoring='accuracy')
y_pred_new = lr_new.predict(X_test_new)

print(cv_results_new)
print(metrics.accuracy_score(y_test_new,y_pred_new))

[0.71350078 0.73369992 0.72113119 0.72113119 0.72977219]
0.7160804020100503
[0.71193093 0.717989   0.73684211 0.71877455 0.71720346]
0.7167085427135679


In [6]:
#creating a model on duplicate dataframe, validating its score and performing feature selection using RFE to see if the model 
#accuracy can be preserved with lesser features

X1 = dta1.drop(columns=['affair'])
y1 = dta1['affair']
X_train1,X_test1,y_train1,y_test1 = train_test_split(X1,y1,test_size=0.25,random_state=42)
lr1 = LogisticRegression(max_iter=1000)
lr1.fit(X_train1,y_train1)
kf1 = KFold(shuffle=True, n_splits=5)
cv_results1 = cross_val_score(lr1, X1, y1, cv=kf1, scoring='accuracy')
y_pred1 = lr1.predict(X_test1)

print(cv_results1)
print(metrics.accuracy_score(y_test1,y_pred1))

rfe1 = RFE(estimator=lr1,n_features_to_select=20)
rfe1.fit(X1,y1)
X1_new = X1[X1.columns[rfe1.get_support(1)]]
X_train1_new,X_test1_new,y_train1_new,y_test1_new = train_test_split(X1_new,y1,test_size=0.25,random_state=42)
lr_new1 = LogisticRegression(max_iter=1000)
lr_new1.fit(X_train1_new,y_train1_new)
kf_new1 = KFold(shuffle=True, n_splits=5)
cv_results_new1 = cross_val_score(lr_new1, X1_new, y1, cv=kf_new1, scoring='accuracy')
y_pred_new1 = lr_new1.predict(X_test1_new)

print(cv_results_new1)
print(metrics.accuracy_score(y_test1_new,y_pred_new1))

[0.73626374 0.73684211 0.72348782 0.71720346 0.71013354]
0.7273869346733668
[0.72762951 0.71720346 0.7282011  0.73919874 0.71327573]
0.7305276381909548
